In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
!pip install numpy requests nlpaug
!pip install torch>=1.6.0 transformers>=4.0.0 sentencepiece
!pip install simpletransformers>=0.61.10

     |████████████████████████████████| 405 kB 5.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="insert",aug_max=10,device='cuda')

In [5]:
data=pd.read_csv('/content/drive/MyDrive/yolov5/New_train1.csv')

In [6]:
data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,clean_comment_text,Language,word_count,char_count,sentence_count,average_word_length,avg_sentence_length,Sentiment,Sentiment_Label
0,0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...,en,23,134,1,5.826087,23.0,0.136364,positive
1,1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,aww match background colour seemingly stuck th...,en,10,58,1,5.800000,10.0,0.250000,positive
2,2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really trying edit war guy constantly ...,en,21,121,1,5.761905,21.0,0.150000,positive
3,3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,make real suggestion improvement wondered sect...,en,52,313,1,6.019231,52.0,0.257143,positive
4,4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page,en,5,25,1,5.000000,5.0,0.000000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159566,159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,second time asking view completely contradicts...,en,23,156,1,6.782609,23.0,0.220000,positive
159567,159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,ashamed horrible thing put talk page,en,6,31,1,5.166667,6.0,-1.000000,negative
159568,159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,spitzer umm there actual article prostitution ...,en,9,57,1,6.333333,9.0,0.000000,neutral
159569,159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,look like actually put speedy first version de...,en,9,48,1,5.333333,9.0,0.125000,positive


In [20]:
abuse_data=data[data['abuse']==1]
abuse_data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,clean_comment_text,Language,word_count,char_count,sentence_count,average_word_length,avg_sentence_length,Sentiment,Sentiment_Label
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker piss around work,en,4,24,1,6.000000,4.0,0.000000e+00,neutral
42,42,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,gay antisemmitian archangel white tiger meow g...,en,61,334,1,5.475410,61.0,6.916667e-02,positive
43,43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,fuck filthy mother as dry,en,5,21,1,4.200000,5.0,-4.222222e-01,negative
55,55,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0,stupid peace shit stop deleting stuff asshole ...,en,13,58,1,4.461538,13.0,-5.000000e-01,negative
56,56,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0,tony sidaway obviously fistfuckee love arm as,en,7,39,1,5.571429,7.0,2.500000e-01,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159494,159494,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1,previous conversation fucking shit eating libe...,en,173,880,1,5.086705,173.0,-3.482792e-01,negative
159514,159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0,mischievious pubic hair,en,3,21,1,7.000000,3.0,0.000000e+00,neutral
159541,159541,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0,absurd edits absurd edits great white shark to...,en,30,184,1,6.133333,30.0,6.938894e-18,positive
159546,159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0,hey listen ever delete edits ever annoyed wwe ...,en,46,222,1,4.826087,46.0,-3.000000e-01,negative


In [ ]:
def threat(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.threat==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = aug.augment(text,n=31)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'threat':1})
    frames=[df,new]
    df=pd.concat(frames)
    df=df.fillna(0)
    return df

In [10]:
! pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.5 MB/s 


In [11]:
! pip install transformers

In [12]:
! pip install SentencePiece

In [13]:

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [14]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
df=threat_data['clean_comment_text'][0:20]

In [ ]:
for i in df:
  print(get_response(i,5))

In [16]:
def highly_malignant1(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.highly_malignant==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = get_response(text,9)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'highly_malignant':1})
    return new

In [17]:
highly_malignant=highly_malignant1(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1595 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3402: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [23]:
highly_malignant['clean_comment_text'][14350:]

14350    Tell anyone that rape jew family choke bagel, ...
14351    Tell anyone that rape jew family choke bagel, ...
14352    Tell anyone that rape jew family choke bagel, ...
14353    Tell anyone that rape jew family choke bagel, ...
14354    Tell anyone that rape jew family choke bagel a...
Name: clean_comment_text, dtype: object

In [19]:
highly_malignant.to_csv("highly_malignant.csv")

In [24]:
highly_malignant['clean_comment_text']

0                           The cocksucker is around work.
1                               cocksucker is around work.
2                                   cocksucker is at work.
3                               The cocksucker is at work.
4                                cocksucker is around work
                               ...                        
14350    Tell anyone that rape jew family choke bagel, ...
14351    Tell anyone that rape jew family choke bagel, ...
14352    Tell anyone that rape jew family choke bagel, ...
14353    Tell anyone that rape jew family choke bagel, ...
14354    Tell anyone that rape jew family choke bagel a...
Name: clean_comment_text, Length: 14355, dtype: object

In [25]:
highly_malignant.drop_duplicates()

,clean_comment_text,highly_malignant
0,The cocksucker is around work.,1
1,cocksucker is around work.,1
2,cocksucker is at work.,1
3,The cocksucker is at work.,1
4,cocksucker is around work,1
...,...,...
14350,"Tell anyone that rape jew family choke bagel, ...",1
14351,"Tell anyone that rape jew family choke bagel, ...",1
14352,"Tell anyone that rape jew family choke bagel, ...",1
14353,"Tell anyone that rape jew family choke bagel, ...",1


In [28]:
def abuse(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.abuse==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = get_response(text,1)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'abuse':1})
    return new

In [ ]:
abuse=abuse(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/7877 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3402: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
abuse

In [ ]:
abuse.to_csv("/content/drive/MyDrive/yolov5/abuse.csv")